### Евгений Шенк

## Лабораторная раота №1. Расчет рейтингов фильмов – RDD

In [1]:
# Params
film_id_for_calc = '318'

In [2]:
import json
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


### Spark Session

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("ESShenk_spark_session")
         .getOrCreate())

### Data

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [5]:
!hdfs dfs -cat /labs/laba01/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [6]:
# Загрузка
rddFilmScores = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.data")
rddFilmNames = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.item")

In [7]:
# Начальная обработка
rddFilmScores = rddFilmScores.map(lambda x: x.split("\t"))
rddFilmNames = rddFilmNames.map(lambda x: x.split("|"))

In [8]:
rddFilmScores.top(5)  # user id | item id | rating | timestamp

[['99', '98', '5', '885679596'],
 ['99', '978', '3', '885679382'],
 ['99', '975', '3', '885679472'],
 ['99', '963', '3', '885679998'],
 ['99', '931', '2', '886780147']]

In [9]:
rddFilmNames.top(1)   # movie id | movie title | release date | video release date |
                      # IMDb URL | unknown | Action | Adventure | Animation |
                      # Children's | Comedy | Crime | Documentary | Drama | Fantasy |
                      # Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
                      # Thriller | War | Western

[['999',
  'Clean Slate (1994)',
  '01-Jan-1994',
  '',
  'http://us.imdb.com/M/title-exact?Clean%20Slate%20(1994)',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [10]:
rddFilmNames.filter(lambda x: x[0] == film_id_for_calc).collect()

[['318',
  "Schindler's List (1993)",
  '01-Jan-1993',
  '',
  "http://us.imdb.com/M/title-exact?Schindler's%20List%20(1993)",
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0']]

### Преобразования

In [11]:
rddFilmScores.filter(lambda x: x[1] == film_id_for_calc).collect()

[['200', '318', '5', '884128458'],
 ['234', '318', '4', '892078890'],
 ['207', '318', '5', '877124871'],
 ['161', '318', '3', '891170824'],
 ['125', '318', '5', '879454309'],
 ['223', '318', '4', '891550711'],
 ['221', '318', '5', '875245690'],
 ['276', '318', '5', '874787496'],
 ['59', '318', '5', '888204349'],
 ['38', '318', '3', '892430071'],
 ['326', '318', '5', '879875612'],
 ['13', '318', '3', '882139686'],
 ['303', '318', '5', '879466523'],
 ['313', '318', '4', '891013712'],
 ['177', '318', '4', '880130618'],
 ['43', '318', '5', '875975717'],
 ['65', '318', '5', '879217689'],
 ['92', '318', '2', '875653307'],
 ['185', '318', '4', '883524172'],
 ['64', '318', '4', '889737593'],
 ['3', '318', '4', '889237482'],
 ['299', '318', '4', '877880649'],
 ['263', '318', '5', '891298453'],
 ['322', '318', '4', '887314280'],
 ['87', '318', '4', '879877627'],
 ['271', '318', '5', '885848707'],
 ['198', '318', '4', '884207560'],
 ['393', '318', '3', '887745973'],
 ['85', '318', '4', '879453684

In [12]:
hist_film = rddFilmScores.filter(lambda x: x[1] == film_id_for_calc)\
                                .map(lambda x: (x[2], 1))\
                                .reduceByKey(lambda x, y: x + y)\
                                .sortByKey()
hist_film.collect()

[('1', 4), ('2', 6), ('3', 23), ('4', 79), ('5', 186)]

In [13]:
hist_all = rddFilmScores.map(lambda x: (x[2], 1))\
                            .reduceByKey(lambda x, y: x + y)\
                            .sortByKey()
hist_all.collect()

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [14]:
result = {
    "hist_film": [x[1] for x in hist_film.collect()],
    "hist_all": [x[1] for x in hist_all.collect()],
}

In [15]:
result

{'hist_film': [4, 6, 23, 79, 186],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

### Сохранить и выйти

In [16]:
with open ("../lab01.json", "w") as file:
    json.dump(result, file)

In [17]:
spark.stop()